## Year on year seasonal trend

In [1]:
import numpy as np
import pandas as pd

In [4]:
# Read data

df=pd.read_table('../SPX_20071123.txt')
df['Date']=df['Date'].round().astype('int')
df['Date']=pd.to_datetime(df['Date'], format='%Y%m%d')
df.set_index('Date', inplace=True)

print(df.head())

                  A       AA     AAPL     ABC  ABI      ABK      ABT    ACAS  \
Date                                                                           
1999-11-24  41.1250  20.3772  23.6563  5.9980  NaN  31.1549  22.8971  7.6786   
1999-11-26  41.1875  20.6142  23.7656  5.9980  NaN  31.1183  22.5998  7.6320   
1999-11-29  42.0000  20.5352  23.6406  5.9675  NaN  30.6418  23.0086  7.5855   
1999-11-30  42.1875  20.6932  24.4688  6.0284  NaN  31.9613  22.5998  7.4925   
1999-12-01  43.0000  21.1078  25.7656  6.0588  NaN  33.5740  22.2652  7.5390   

                ACE      ACS  ...      XEL       XL     XLNX      XOM  \
Date                          ...                                       
1999-11-24  12.8492  19.0000  ...  14.8614  38.5249  43.8110  35.1539   
1999-11-26  12.8492  18.7188  ...  14.9079  38.5249  43.5356  34.6541   
1999-11-29  12.2246  18.7188  ...  14.8614  38.2478  46.8092  35.2650   
1999-11-30  12.1353  18.7188  ...  15.0926  37.6935  43.8110  35.2372   
1

In [3]:
eomPrice=df.resample('M').last()[:-1] # End of month prices.  Need to remove last date because it isn't really end of January.
monthlyRet=eomPrice.pct_change(1, fill_method=None)

# Create a dataframe of positions.  Each row is a date, each column is a ticker.
positions=np.zeros(monthlyRet.shape)

print(monthlyRet.head())
print(positions.shape)


                   A        AA      AAPL       ABC  ABI       ABK       ABT  \
Date                                                                          
1999-11-30       NaN       NaN       NaN       NaN  NaN       NaN       NaN   
1999-12-31  0.832593  0.267180  0.050444  0.227274  NaN -0.042433 -0.044412   
2000-01-31 -0.143897 -0.160396  0.009120  0.193418  NaN -0.062277 -0.087613   
2000-02-29  0.566572 -0.000831  0.104821 -0.196546  NaN -0.093299  0.005755   
2000-03-31  0.003014  0.029198  0.184839  0.030039  NaN  0.146513  0.082064   

                ACAS       ACE       ACS  ...       XEL        XL      XLNX  \
Date                                      ...                                 
1999-11-30       NaN       NaN       NaN  ...       NaN       NaN       NaN   
1999-12-31  0.195916  0.003280  0.228711  ... -0.028252  0.017157  0.016060   
2000-01-31  0.137360  0.059926 -0.135870  ... -0.012825 -0.125305  0.006187   
2000-02-29 -0.072464  0.010601 -0.207547  ... -0.08

In [5]:
# Loop through each month and create positions, which is a matrix of 0, 1, or -1.

for m in range(13, monthlyRet.shape[0]):
    hasData=np.where(np.isfinite(monthlyRet.iloc[m-12, :]))[0]
    sortidx=np.argsort(monthlyRet.iloc[m-12, hasData])  
    badData=np.where(np.logical_not(np.isfinite(monthlyRet.iloc[m-1, hasData[sortidx]])))[0] # these are indices
    sortidx.drop(sortidx.index[badData], inplace=True)
    topN=np.floor(len(sortidx)/10).astype('int')
    positions[m-1, hasData[sortidx.values[np.arange(0, topN)]]]=-1
    positions[m-1, hasData[sortidx.values[np.arange(-topN,0)]]]=1


In [6]:
# Calculate capital
capital=np.nansum(np.array(pd.DataFrame(abs(positions)).shift()), axis=1) # capital is the sum of the absolute value of positions
capital[capital==0]=1 # avoid divide by zero

# Calculate returns
ret=np.nansum(np.array(pd.DataFrame(positions).shift())*np.array(monthlyRet), axis=1)/capital
ret=np.delete(ret, np.arange(13))

avgret=np.nanmean(ret)*12 # annualized average return
sharpe=np.sqrt(12)*np.nanmean(ret)/np.nanstd(ret)

In [8]:
# Calculate drawdowns
dd=np.zeros(ret.shape)
for i in range(1, len(ret)):
    dd[i]=np.nanmin(ret[:i])-ret[i]
maxdd=np.nanmax(dd)

print('Avg ann return=%f Sharpe ratio=%f' % (avgret, sharpe))
print('Max drawdown=%f' % maxdd)


Avg ann return=-0.012679 Sharpe ratio=-0.122247
Max drawdown=0.060729
